<a href="https://colab.research.google.com/github/AchrafAsh/awesome-pytorch-notebooks/blob/main/01_machine_translation_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation with Transformers

<img src="https://pytorch.org/tutorials/_images/transformer_architecture.jpg" width="400px" />

Official Pytorch Tutorial: https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html

In [1]:
import io
import os
import pandas as pd
import gensim

import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import Counter

from torch.nn.utils.rnn import pad_sequence  # padding of every batch
from torch.utils.data import Dataset, DataLoader

from torchtext.data.utils import get_tokenizer
from torchtext.utils import download_from_url, extract_archive
from torchtext.vocab import Vocab

from typing import List

## The Data

### Dataset and Data processing

In [2]:
!python -m spacy download en
!python -m spacy download fr

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [3]:
fr_tokenizer = get_tokenizer('spacy', language='fr')
en_tokenizer = get_tokenizer('spacy', language='en')

In [4]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.fr.gz', 'train.en.gz')
val_urls = ('val.fr.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.fr.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

In [5]:
def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            counter.update(tokenizer(string_))
    return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

In [ ]:
fr_vocab = build_vocab(train_filepaths[0], fr_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

In [ ]:
def data_process(filepaths):
    raw_fr_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_fr, raw_en) in zip(raw_fr_iter, raw_en_iter):
        fr_tensor_ = torch.tensor([fr_vocab[token] for token in fr_tokenizer(raw_fr)],
                                dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                                dtype=torch.long)
        data.append((fr_tensor_, en_tensor_))
    return data

In [ ]:
train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

### Data Loader

In [ ]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

In [ ]:
def generate_batch(data_batch):
    fr_batch, en_batch = [], []
    for (fr_item, en_item) in data_batch:
        fr_batch.append(torch.cat([torch.tensor([BOS_IDX]), fr_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    return fr_batch, en_batch

In [ ]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

## The model

- Embeddings → encode one-hot-encoded words as continuous vectors to catch semantic (might use pre-trained word2vec for that)
- Transformer Block stacked

In [ ]:
class Translator(nn.Module):
    def __init__(self, 
                 src_vocab_size: int,
                 trgt_vocab_size: int,
                 hidden_dim:int=124,
                 word_vectors=None):
        
        super().__init__()
        if word_vectors not None:
            self.embedding = nn.Embedding.from_pretrained(weight)
            assert hidden_dim == weight.size(0) # TODO: update to make sure the output is the same as the input of the transformer
        else:
            self.embedding = nn.Embedding(num_embeddings=src_vocab_size,
                                          embedding_dim=hidden_dim)
        
        self.transformer = nn.Transformer(d_model=hidden_dim, nhead=8) # docs: https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html
    
    def load_embeddings(self, keyed_vectors):
        self.embedd

    def forward(self, src, tgt):
        src, tgt = self.embedding(src), self.embedding(tgt)
        return self.transformer(src, tgt)

## The Training

### Optional: pre-trained embeddings

In [ ]:
# TODO: download a pre-trained word2vec (a very small one to see if a pre-trained yield better results)
!wget 

In [ ]:
# Load pre-trained word vectors
model = gensim.models.KeyedVectors.load_word2vec_format('path/to/file')
weights = torch.FloatTensor(model.vectors)

### Utility functions

In [ ]:
def run(model:nn.Module,
        iterator: DataLoader,
        epochs: int,
        lr:float=0.01,
        weight_decay:float=0.001):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(1, epochs+1):
        total_loss = 0
        for idx, data in enumerate(dataset):
            src, tgt = zip(data)
            loss = train(model, src, tgt, optimizer)
            total_loss += loss

        print(f"Epoch: [{epoch} / {epochs}] | Loss: {total_loss}")


def train(model, src, tgt, optimizer):
    PAD_IDX = en_vocab.stoi['<pad>']

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

    model.train()
    output = model(src, tgt)
    loss = nn.CrossEntropyLoss()(output, tgt[])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss
    

def evaluate(model, dataset):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    accuracy = 0

    for idx, data in enumerate(dataset):
        src, tgt = zip(data)
        output = model(src, tgt)
        loss = criterion(output, tgt)
        total_loss += loss
    
    print(f"Total Loss: {total_loss}")

In [ ]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

In [ ]:
model = Translator()

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"The model has {count_parameters(model):,} parameters 🚀")

In [ ]:
run(model, train_iter, epochs=EPOCHS)

# Building the Transformer from scratch

- Attention is all you need: [link to paper]

[image of the architecture]

## Self-Attention

In [ ]:
class SelfAttention(nn.Module):
	def __init__(self, k, heads=8):
		super(SelfAttention, self).__init__()
		self.k, self.heads = k, heads

		# these compute the queries, keys, values for all
		# heads (as a single concatenated vector)
		self.tokeys = nn.Linear(k, k * heads, bias=False)
		self.toqueries = nn.Linear(k, k * heads, bias=False)
		self.tovalues = nn.Linear(k, k * heads, bias=False)

		# this unifies the outputs of the different heads into a single k-vector
		self.unifyheads = nn.Linear(heads * k, k)

	def forward(self, x):
		b, t, k = x.size()
		h = self.heads

		queries = self.toqueries(x).view(b, t, h, k)
		keys = self.tokeys(x).view(b, t, h, k)
		values = self.values(x).view(b, t, h, k)
		# - fold heads into the batch dimension
		keys = keys.transpose(1, 2).contiguous().view(b * h, t, k)
		queries = queries.transpose(1, 2).contiguous().view(b * h, t, k)
		values = values.transpose(1, 2).contiguous().view(b * h, t, k)

		queries = queries / (k ** (1/4))
		keys = keys / (k ** (1/4))

		# - get dot product of queries and keys, and scale
		dot = torch.bmm(queries, keys.transpose(1, 2))
		# - dot has size (b*h, t, t) containing raw weights

		dot = F.softmax(dot, dim=2)
		# - dot now contains row-wise normalized weights

		# apply the self attention to the values
		out = torch.bmm(dot, values).view(b, h, t, k)

		# - swap h, t back  and unify heads
		out = out.transpose(1, 2).contiguous().view(b, t, h * k)
		return self.unifyheads(out)

## Transformer Block

In [ ]:
class TransformerBlock(nn.Module):
	def __init__(self, k, heads):
		super().__init__()

		self.attention = SelfAttention(k, heads=heads)

		self.norm1 = nn.LayerNorm(k)
		self.norm2 = nn.LayerNorm(k)

		self.ff = nn.Sequential(
			nn.Linear(k, 4*k),
			nn.ReLU(),
			nn.Linear(4*k, k)
		)

	def forward(self, x):
		attended = self.attention(x)
		x = self.norm1(attended + x)

		fedforward = self.ff(x)
		return self.norm2(fedforward + x)

## Encoder / Decoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self):

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()

## Putting everything together

In [ ]:
class Transformer(nn.Module):
	def __init__(self, k, heads, depth, seq_length, num_tokens, num_classes):
		super().__init__()

		self.num_tokens = num_tokens
		self.token_emb = nn.Embedding(num_tokens, k)
		self.pos_emb = nn.Embedding(seq_length, k)

		# the sequence of transformer blocks that does all the heavy lifting
		tblocks = []
		for i in range(depth):
			tblocks.append(TransformerBlock(k=k, heads=heads))
		self.tblocks = nn.Sequential(*tblocks)

		# maps the final output sequence to class logits
		self.toprobs = nn.Linear(k, num_classes)

	def forward(self, x):
		"""
		:param x: A (b,t) tensor of integer values representing words 
				(in some predetermined vocabulary)
		:return: A (b, c) tensor of log-probabilities over the classes
				(where c is the nr. of classes)
		"""

		# generate token embeddings
		tokens = self.token_emb()
		b, t, k = tokens.size()

		# generate position embeddings
		positions = torch.arange(t)
		positions = self.pos_emb(positions)[None, :, :].expand(b, t, k)

		x = tokens + positions
		x = self.tblocks(x)

		# average-pool over the t dimension and project to class probabilities
		x = self.toprobs(x.mean(dim=1))
		return F.log_softmax(x, dim=1)